# Q2: Data Cleaning

**Phase 3:** Data Cleaning & Preprocessing  
**Points: 9 points**

**Focus:** Handle missing data, outliers, validate data types, remove duplicates.

**Lecture Reference:** Lecture 11, Notebook 1 ([`11/demo/01_setup_exploration_cleaning.ipynb`](https://github.com/christopherseaman/datasci_217/blob/main/11/demo/01_setup_exploration_cleaning.ipynb)), Phase 3. Also see Lecture 05 (data cleaning).

---

## Setup

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Load data from Q1 (or directly from source)
df = pd.read_csv('data/beach_sensors.csv')
# If you saved cleaned data from Q1, you can load it:
# df = pd.read_csv('output/q1_exploration.csv')  # This won't work - load original

---

## Objective

Clean the dataset by handling missing data, outliers, validating data types, and removing duplicates.

**Time Series Note:** For time series data, forward-fill (`ffill()`) is often appropriate for missing values since sensor readings are continuous. However, you may choose other strategies based on your analysis.

---

## Required Artifacts

You must create exactly these 3 files in the `output/` directory:

### 1. `output/q2_cleaned_data.csv`
**Format:** CSV file
**Content:** Cleaned dataset with same structure as original (same columns)
**Requirements:**
- Same columns as original dataset
- Missing values handled (filled, dropped, or imputed)
- Outliers handled (removed, capped, or transformed)
- Data types validated and converted
- Duplicates removed
- **Sanity check:** Dataset should retain most rows after cleaning (at least 1,000 rows). If you're removing more than 50% of data, reconsider your strategy—imputation is usually preferable to dropping rows for this dataset.
- **No index column** (save with `index=False`)

### 2. `output/q2_cleaning_report.txt`
**Format:** Plain text file
**Content:** Detailed report of cleaning operations
**Required information:**
- Rows before cleaning: [number]
- Missing data handling method: [description]
  - Which columns had missing data
  - Method used (drop, forward-fill, impute, etc.)
  - Number of values handled
- Outlier handling: [description]
  - Detection method (IQR, z-scores, domain knowledge)
  - Which columns had outliers
  - Method used (remove, cap, transform)
  - Number of outliers handled
- Duplicates removed: [number]
- Data type conversions: [list any conversions]
- Rows after cleaning: [number]

**Example format:**
```
DATA CLEANING REPORT
====================

Rows before cleaning: 50000

Missing Data Handling:
- Water Temperature: 2500 missing values (5.0%)
  Method: Forward-fill (time series appropriate)
  Result: All missing values filled
  
- Air Temperature: 1500 missing values (3.0%)
  Method: Forward-fill, then median imputation for remaining
  Result: All missing values filled

Outlier Handling:
- Water Temperature: Detected 500 outliers using IQR method (3×IQR)
  Method: Capped at bounds [Q1 - 3×IQR, Q3 + 3×IQR]
  Bounds: [-5.2, 35.8]
  Result: 500 values capped

Duplicates Removed: 0

Data Type Conversions:
- Measurement Timestamp: Converted to datetime64[ns]

Rows after cleaning: 50000
```

### 3. `output/q2_rows_cleaned.txt`
**Format:** Plain text file
**Content:** Single integer number (total rows after cleaning)
**Requirements:**
- Only the number, no text, no labels
- No whitespace before or after
- Example: `50000`

---

## Requirements Checklist

- [ ] Missing data handling strategy chosen and implemented
- [ ] Outliers detected and handled (IQR method, z-scores, or domain knowledge)
- [ ] Data types validated and converted
- [ ] Duplicates identified and removed
- [ ] Cleaning decisions documented in report
- [ ] All 3 required artifacts saved with exact filenames

---

## Your Approach

1. **Handle missing data** - Choose appropriate strategy (drop, forward-fill, impute) based on data characteristics
2. **Detect and handle outliers** - Use IQR method or z-scores; decide whether to remove, cap, or transform
3. **Validate data types** - Ensure numeric and datetime columns are properly typed
4. **Remove duplicates**
5. **Document and save** - Write detailed cleaning report explaining your decisions

---

## Decision Points

- **Missing data:** Should you drop rows, impute values, or forward-fill? Consider: How much data is missing? Is it random or systematic? For time series, forward-fill is often appropriate.
- **Outliers:** Are they errors or valid extreme values? Use IQR method or z-scores to detect, then decide: remove, cap, or transform. Document your reasoning.
- **Data types:** Are numeric columns actually numeric? Are datetime columns properly formatted? Convert as needed.

---

## Checkpoint

After Q2, you should have:
- [ ] Missing data handled
- [ ] Outliers addressed
- [ ] Data types validated
- [ ] Duplicates removed
- [ ] All 3 artifacts saved: `q2_cleaned_data.csv`, `q2_cleaning_report.txt`, `q2_rows_cleaned.txt`

---

**Next:** Continue to `q3_data_wrangling.md` for Data Wrangling.


## Part 1:

In [4]:
clean_df = pd.read_csv('data/beach_sensors.csv')
rows_before = clean_df.shape[0]

clean_df["Measurement Timestamp"] = pd.to_datetime(clean_df["Measurement Timestamp"], errors="coerce")
numeric_cols = clean_df.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = clean_df.select_dtypes(include=['object']).columns

missing_before = clean_df.isna().sum()

##Missing value:
for col in numeric_cols:
    clean_df[col] = clean_df[col].fillna(clean_df[col].median())
for col in categorical_cols:
    clean_df[col] = clean_df[col].fillna("Unknown")

# record outliers before cleaning
outliers_before = {}
for col in numeric_cols:
    Q1 = clean_df[col].quantile(0.25)
    Q3 = clean_df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    count = ((clean_df[col] < lower) | (clean_df[col] > upper)).sum()
    outliers_before[col] = int(count)

##outliers handling
for col in numeric_cols:
    Q1 = clean_df[col].quantile(0.25)
    Q3 = clean_df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    clean_df[col] = clean_df[col].clip(lower, upper)

##no duplicates, so part 2 I can just do the text report
rows_before_dedup = clean_df.shape[0]
clean_df = clean_df.drop_duplicates()
duplicates_removed = rows_before_dedup - clean_df.shape[0]

rows_after = clean_df.shape[0]

##save
clean_df.to_csv("output/q2_cleaned_data.csv", index=False)
print("Saved to output/q2_cleaned_data.csv")
print("Final shape:", clean_df.shape)


Saved to output/q2_cleaned_data.csv
Final shape: (196526, 18)


## Part 2:

In [8]:
report = ""

report += "DATA CLEANING REPORT\n"
report += "====================\n\n"

report += f"Rows before cleaning: {rows_before}\n\n"

#Missing Data
report += "Missing Data Handling:\n"
for col in clean_df.columns:
    before = int(missing_before[col])
    if before > 0:
        if col in numeric_cols:
            method = "Filled with median"
        else:
            method = "Filled with 'Unknown'"
        report += f"- {col}: {before} missing values\n"
        report += f"  Method: {method}\n\n"

if missing_before.sum() == 0:
    report += "- No missing values found.\n\n"

#Outliers
report += "Outlier Handling:\n"
any_outliers = False
for col in numeric_cols:
    count = outliers_before[col]
    if count > 0:
        any_outliers = True
        report += f"- {col}: {count} outliers detected (IQR method)\n"
        report += "  Method: Capped using IQR bounds\n\n"

if not any_outliers:
    report += "- No numeric outliers detected.\n\n"

#Duplicate
report += f"Duplicates Removed: {duplicates_removed}\n\n"

#Data type
report += "Data Type Conversions:\n"
report += "- Measurement Timestamp converted to datetime\n\n"

report += f"Rows after cleaning: {rows_after}\n"
print(report)

#Save
with open("output/q2_cleaning_report.txt", "w") as f:
    f.write(report)
print("yeah saved! output/q2_cleaning_report.txt")


DATA CLEANING REPORT

Rows before cleaning: 196526

Missing Data Handling:
- Air Temperature: 75 missing values
  Method: Filled with median

- Wet Bulb Temperature: 76054 missing values
  Method: Filled with median

- Rain Intensity: 76054 missing values
  Method: Filled with median

- Total Rain: 76054 missing values
  Method: Filled with median

- Precipitation Type: 76054 missing values
  Method: Filled with median

- Barometric Pressure: 146 missing values
  Method: Filled with median

- Heading: 76054 missing values
  Method: Filled with median

Outlier Handling:
- Air Temperature: 97 outliers detected (IQR method)
  Method: Capped using IQR bounds

- Wet Bulb Temperature: 13308 outliers detected (IQR method)
  Method: Capped using IQR bounds

- Humidity: 185 outliers detected (IQR method)
  Method: Capped using IQR bounds

- Rain Intensity: 4250 outliers detected (IQR method)
  Method: Capped using IQR bounds

- Interval Rain: 15862 outliers detected (IQR method)
  Method: Cappe

## Part 3:

In [10]:
rows_after = clean_df.shape[0]
print(rows_after)

with open("output/q2_rows_cleaned.txt", "w") as f:
    f.write(str(rows_after))

print("Saved to output/q2_rows_cleaned.txt")

196526
Saved to output/q2_rows_cleaned.txt
